<a href="https://colab.research.google.com/github/sisisa/research_UsedCode/blob/main/1all_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
# !pip install

Cloning into 'transformers'...
remote: Enumerating objects: 170709, done.
remote: Counting objects: 100% (26465/26465), done.
remote: Compressing objects: 100% (1661/1661), done.
remote: Total 170709 (delta 25690), reused 24824 (delta 24798), pack-reused 144244
Receiving objects: 100% (170709/170709), 168.98 MiB | 15.17 MiB/s, done.
Resolving deltas: 100% (129686/129686), done.
Updating files: 100% (3805/3805), done.
/content/transformers


In [ ]:
!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3
!pip install fugashi ipadic
# !pip install pytorch-lightning

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig4.0
Suggested packages:
  apt-xapian-index aptitude-doc-en | aptitude-doc debtags tasksel libcwidget-dev xapian-tools
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcwidget4 libsigc++-2.0-0v5 libxapian30 swig swig4.0
0 upgraded, 7 newly installed, 0 to remove and 9 not upgraded.
Need to get 4,954 kB of archives.
After this operation, 22.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aptitude-common all 0.8.13-3ubuntu1 [1,719 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsigc++-2.0-0v5 amd64 2.10.4-2ubuntu3 [12.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcwidget4 amd64 0.5.18-5build1 [306 kB]
Get:4

In [ ]:
import torch
# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
""" ライブラリのインポート """
import pandas as pd

file = '/content/sample_data/all_data.csv'
print(file)
df = pd.read_csv(file, encoding='utf-8')

print(df.head())
# データの確認
print(f'データサイズ： {df.shape}')
df.dtypes

/content/sample_data/all_data.csv
                                                ツイート  ラベル
0                 SMのLINEオーディション\n追加資料の返信来られた方いますかね   0.0
1  名前さなだ\n好感度010065\n告白されたらまだ\n寝落ちできるしてみたい\n彼氏彼女い...  0.0
2    \n100万人におつかれ生です\nキャンペーン開始\n1日1回その場でチャレンジ\n\n...  0.0
3  ご参加ありがとうございます\n\n抽選結果は下記ページからご確認いただけます\n当選者には後...  0.0
4  LINE前のアカウント削除でなくなってしまい確認取れないので無理\n\nあと買った家でこの方...  1.0
データサイズ： (7200, 2)


ツイート     object
ラベル     float64
dtype: object

In [ ]:
# データの抽出
sentences = df['ツイート'].values
# df['ラベル'] = df['ラベル'].astype(int)
# df.dtypes
labels = df['ラベル'].values


# Tensor 形式へ変換
# x = torch.tensor(sentences.values, dtype=torch.float32)
# t = torch.tensor(labels.values, dtype=torch.int64)

# # 結果の表示
print("Sentences:", sentences[0])
print("Labels:", labels[0])

Sentences: SMのLINEオーディション
追加資料の返信来られた方いますかね 
Labels: 0.0


In [ ]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
# Tokenizerの準備
from transformers import BertJapaneseTokenizer, BertForMaskedLM
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# テスト実行
# 元文章
print(' Original: ', sentences[0])
# Tokenizer：単語単位で分割された文章
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
# Token-id：単語をIDに置き換えた文章
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))


 Original:  SMのLINEオーディション
追加資料の返信来られた方いますかね 
Tokenized:  ['SM', 'の', 'LINE', 'オーディション', '追加', '資料', 'の', '返', '##信', '来', '##ら', 'れ', 'た', '方', 'い', 'ます', 'か', 'ね']
Token IDs:  [6838, 5, 21117, 6299, 2143, 2710, 5, 2434, 28836, 1276, 28469, 20, 10, 283, 21, 2610, 29, 1852]


In [ ]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数')


最大単語数:  111
上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値が最大単語数


In [ ]:
# 配列の初期化(input_ids,attention_masks)
input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True, # Special Tokenの追加
                        max_length = 113,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得
    input_ids.append(encoded_dict['input_ids'])

    # Attention_maskの取得
    attention_masks.append(encoded_dict['attention_mask'])


# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

print(input_ids.dim())
print(attention_masks.dim())
# tensor型に変換
labels = torch.tensor(labels)
# print(labels.size())
# one-hot エンコーディングに変換
# labels_onehot = torch.nn.functional.one_hot(labels, num_classes=3)
# torch.unsqueeze(labels, 1)
# print(labels)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

print(input_ids.shape)
print(attention_masks.shape)
print(labels.shape)
# print('One-hot Labels:', labels_onehot[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


2
2
Original:  SMのLINEオーディション
追加資料の返信来られた方いますかね 
Token IDs: tensor([    2,  6838,     5, 21117,  6299,  2143,  2710,     5,  2434, 28836,
         1276, 28469,    20,    10,   283,    21,  2610,    29,  1852,     3,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0])
torch.Size([7200, 113])
torch.Size([7200, 113])
torc

In [ ]:
# データセットの分割とデータローダーの作成
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# データを6:2:2に分割
train_data = int(0.6 * len(dataset))# 学習データ
val_data = int(0.2 * len(dataset))# 検証データ
# len(dataset) - train_data
test_data = int(0.2 * len(dataset))# テストデータ

# データセットを分割
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_data, val_data, test_data])

print('学習データ数：{}'.format(train_data))
print('検証データ数:　{} '.format(val_data))
print('テストデータ数:　{} '.format(test_data))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            dataset=train_dataset,
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            dataset=val_dataset,
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )
# テストデータローダー
# test_dataloader = DataLoader(
#             test_dataset,
#             sampler = SequentialSampler(test_dataset), # 順番にデータを取得してバッチ化
#             batch_size = batch_size
#         )


学習データ数：4320
検証データ数:　1440 
テストデータ数:　1440 


In [ ]:
# !pip install accelerate -U

# # 評価用の関数(sklearn)
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# def compute_metrics(result):
#     labels = result.label_ids
#     preds = result.predictions.argmax(-1)
#     #2値分類ならaverage='binary'とする
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
#     acc = accuracy_score(labels, preds)
#     return {
#         'accuracy': acc,
#         'precision': precision,
#         'recall': recall,
#         'f1': f1,
#     }

# from transformers import TrainingArguments

# training_args = TrainingArguments(
#     output_dir='./content/sample_data/results',
#     evaluation_strategy='steps',
#     metric_for_best_model='f1',
#     per_device_train_batch_size=32,
#     per_device_eval_batch_size=32,
#     num_train_epochs=3,
#     warmup_steps=500,
#     weight_decay=0.01
# )

# from transformers import Trainer
# from transformers import EarlyStoppingCallback

# trainer = Trainer(
#     model=model_name,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
# )


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.4 MB/s eta 0:00:00


ImportError: ignored

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model_name = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 東北大学が公開している日本語モデルの指定
    num_labels = 1, # ラベル数
)
model_name.cuda()

# trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# from torch import nn
# # 最適化手法の設定
# optimizer = AdamW(model_name.parameters(), lr=2e-5)

# # 訓練パートの定義
# def train(model_name):
#     model_name.train() # 訓練モードで実行
#     train_loss = 0
#     for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)
#         optimizer.zero_grad()
#         loss, logits = model_name(b_input_ids,
#                              token_type_ids=None,
#                              attention_mask=b_input_mask,
#                              labels=b_labels)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model_name.parameters(), 1.0)
#         optimizer.step()
#         train_loss += loss.item()
#     return train_loss

# # テストパートの定義
# def validation(model_name):
#     model_name.eval()# 訓練モードをオフ
#     val_loss = 0
#     with torch.no_grad(): # 勾配を計算しない
#         for batch in validation_dataloader:
#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)
#             with torch.no_grad():
#                 (loss, logits) = model_name(b_input_ids,
#                                     token_type_ids=None,
#                                     attention_mask=b_input_mask,
#                                     labels=b_labels)
#             val_loss += loss.item()
#     return val_loss

# テストパートの定義
# def test(model_name):
#     model_name.eval()  # 訓練モードをオフ
#     test_loss = 0
#     with torch.no_grad():
#         for batch in test_dataloader:
#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)
#             with torch.no_grad():
#                 (loss, logits) = model_name(b_input_ids,
#                                     token_type_ids=None,
#                                     attention_mask=b_input_mask,
#                                     labels=b_labels)
#             test_loss += loss.item()
#     return test_loss

# 最適化手法の設定
optimizer = torch.optim.AdamW(model_name.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model_name):
    model_name.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        loss = model_name(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels).loss # 戻り値とここを修正
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_name.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model_name):
    model_name.eval() # 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                loss = model_name(b_input_ids,
                                    token_type_ids=None,
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
            val_loss += loss.item()
    return val_loss

In [ ]:
# 学習の実行
# max_epoch = 4
# train_loss_ = []
# test_loss_ = []

# for epoch in range(max_epoch):
#     train_ = train(model_name)
#     test_ = validation(model_name)
#     train_loss_.append(train_)
#     test_loss_.append(test_)
# # 検証方法の確認（1バッチ分で計算ロジックに確認）

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

In [ ]:
model_name.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
        # 学習済みモデルによる予測結果をpredsで取得
        preds = model_name(b_input_ids,
                      token_type_ids=None,
                      attention_mask=b_input_mask)

In [ ]:
## 予測結果の確認
print(f'出力:{preds}')


出力:SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1097],
        [ 0.1448],
        [-0.1255],
        [ 0.2096],
        [ 0.0312],
        [ 0.2377],
        [ 0.0318],
        [-0.0260],
        [-0.0221],
        [ 0.0970],
        [ 0.0803],
        [ 0.1055],
        [ 0.3398],
        [-0.0418],
        [-0.1700],
        [ 0.0575],
        [ 0.0789],
        [ 0.0908],
        [-0.0754],
        [ 0.0739],
        [ 0.0733],
        [ 0.0285],
        [ 0.0058],
        [ 0.1195],
        [ 0.0412],
        [ 0.0730],
        [ 0.0649],
        [ 0.2677],
        [ 0.2694],
        [ 0.0414],
        [ 0.1537],
        [-0.0246]], device='cuda:0'), hidden_states=None, attentions=None)


In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
import numpy as np

# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1','logit_2'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()


ValueError: ignored